# M/EEG exam instructions
- Assignment format: 
    - mandatory: a notebook with your answers 
    - optional: an additional document with your answers to the conceptual questions
- Please send your assignment to M.Corsi's email address
- Deadline: 
    - **Feb 12th, 8AM.** Please not that an **extension will not be proposed.**

In [ ]:
%matplotlib qt

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from matplotlib.colors import TwoSlopeNorm

import mne
from mne.datasets import eegbci
from mne.io import concatenate_raws, read_raw_edf
from mne.stats import permutation_cluster_1samp_test as pcluster_test
from mne.time_frequency import tfr_multitaper

## Part 1 - Connectivity and Networks

Here is an EEG dataset to load:

In [ ]:
#Define the parameters
subject = 1  # use data from subject 1
runs = [6, 10, 14]  # Motor imagery: hands vs feet

# Extract raw data
fnames = eegbci.load_data(subjects=subject, runs=runs)
raw = concatenate_raws([read_raw_edf(f, preload=True) for f in fnames])
raw.rename_channels(lambda x: x.strip("."))  # remove dots from channel names
events, _ = mne.events_from_annotations(raw, event_id=dict(T1=2, T2=3))
channel_names = raw.info['ch_names']

# Extract trials between -1s and 4s
tmin, tmax = -1, 4
event_ids = dict(hands=2, feet=3)  # map event IDs to tasks
epochs = mne.Epochs(
    raw,
    events,
    event_ids,
    tmin - 0.5,
    tmax + 0.5,
    baseline=None,
    preload=True,
)

## Questions:
- For each condition:
    - Compute and plot the connectivity matrices based on the estimation of imaginary coherence, averaged over the mu band and across epochs. *Question: What do these matrices reveal about potential changes in brain connectivity between the tasks performed by the subject?*
    - Compute and plot the associated node strength, averaged across epochs. *Question: What do these results suggest about potential changes in node strength between the tasks performed by the subject?*

- Below is a plot showing the statistical difference between the MI (Motor Imagery) and Rest conditions, derived from imaginary coherence (left) and node strength (right).
*Questions:*
    - *What do you observe in these plots?*
    - *Are these observations neurophysiologically meaningful? Justify your answer.*
![Figure_ImCoh](MI_Rest_ImCoh.png)

## Part 2 - Features in BCI

Here is an EEG dataset to load:

In [ ]:
#Define the parameters
subject = 1  # use data from subject 1
runs = [6, 10, 14]  # Motor imagery: hands vs feet

# Extract raw data
fnames = eegbci.load_data(subjects=subject, runs=runs)
raw = concatenate_raws([read_raw_edf(f, preload=True) for f in fnames])
raw.rename_channels(lambda x: x.strip("."))  # remove dots from channel names
events, _ = mne.events_from_annotations(raw, event_id=dict(T1=2, T2=3))

# Extract trials between -1s and 4s
channelsOfInterest = "T7", "C3", 'O1' # to get the full list of channels you can type: raw.info['ch_names']
tmin, tmax = -1, 4
event_ids = dict(hands=2, feet=3)  # map event IDs to tasks
epochs = mne.Epochs(
    raw,
    events,
    event_ids,
    tmin - 0.5,
    tmax + 0.5,
    picks=(channelsOfInterest),
    baseline=None,
    preload=True,
)

# Compare power spectra computed in each condition/channel 
freqs = np.arange(2, 45)
vmin, vmax = -1, 1.5  # set min and max ERDS values in plot
baseline = (-1, 0)  # baseline interval (in s)
cnorm = TwoSlopeNorm(vmin=vmin, vcenter=0, vmax=vmax)  # min, center & max ERDS

kwargs = dict(
    n_permutations=100, step_down_p=0.05, seed=1, buffer_size=None, out_type="mask"
)  # for cluster test

# Time-Frequency decomposition
tfr = tfr_multitaper(
    epochs,
    freqs=freqs,
    n_cycles=freqs,
    use_fft=True,
    return_itc=False,
    average=False,
    decim=2,
)
tfr.crop(tmin, tmax).apply_baseline(baseline, mode="percent")

nb_channels = len(channelsOfInterest)
for event in event_ids:
    # select desired epochs for visualization
    tfr_ev = tfr[event]
    fig, axes = plt.subplots(
        1, 4, figsize=(12, 4), gridspec_kw={"width_ratios": [10, 10, 10, 1]}
    )
    for ch, ax in enumerate(axes[:-1]):  # for each channel
        # positive clusters
        _, c1, p1, _ = pcluster_test(tfr_ev.data[:, ch], tail=1, **kwargs)
        # negative clusters
        _, c2, p2, _ = pcluster_test(tfr_ev.data[:, ch], tail=-1, **kwargs)

        # note that we keep clusters with p <= 0.05 from the combined clusters
        # of two independent tests; in this example, we do not correct for
        # these two comparisons
        c = np.stack(c1 + c2, axis=2)  # combined clusters
        p = np.concatenate((p1, p2))  # combined p-values
        mask = c[..., p <= 0.05].any(axis=-1)

        # plot TFR (ERDS map with masking)
        tfr_ev.average().plot(
            [ch],
            cmap="RdBu",
            cnorm=cnorm,
            axes=ax,
            colorbar=False,
            show=False,
            mask=mask,
            mask_style="mask",
        )

        ax.set_title(epochs.ch_names[ch], fontsize=10)
        ax.axvline(0, linewidth=1, color="black", linestyle=":")  # event
        if ch != 0:
            ax.set_ylabel("")
            ax.set_yticklabels("")
    fig.colorbar(axes[0].images[-1], cax=axes[-1]).ax.set_yscale("linear")
    fig.suptitle(f"ERDS ({event})")
    plt.show()

## Questions:
- *Describe the observations you can make from the maps. Are these observations neurophysiologically relevant or meaningful? Justify your answer.*
- *To what extent are these observations informative for assessing or predicting BCI performance?*
- As the experimenter, based on your observations: *Which electrode(s) and frequency bin(s) would you select for feature extraction? Explain why you believe these choices would be optimal for improving BCI performance or interpretability.*


## Part 3 - Machine Learning & BCI

## Context

Here is a publicly available [BCI dataset](http://moabb.neurotechx.com/docs/generated/moabb.datasets.BNCI2014_001.html#moabb.datasets.BNCI2014_001) (cf below to load the data and to get information regarding the experimental information). In the following lines of code we defined two classification pipelines (CSP+LDA: Common Spatial Patterns + LDA, RG+LR:Riemannian Geometry + Logistic Regression), and we plotted their performances from a dataset composed of 2 subjects.

## Questions
- *What key observations can you draw from the plots presented?*
- *Instead of the methods implemented here, propose an alternative framework for feature extraction, selection, and classification.*
*Sub-questions:*
    - *Why would this framework be advantageous?*
    - *How would you assess its performance? Provide at least one evaluation metric beyond classical accuracy to assess (mis-)classification.*
- Alternative pipeline
    - *Implement your proposed framework below and compare its performance to the existing pipelines (RG+LR and CSP+LDA).*
    - *What are your findings? Do you have suggestions to further improve the performance of your framework (e.g., feature engineering, sample selection, or preprocessing)?*


In [ ]:
import warnings

from mne.decoding import CSP
from pyriemann.estimation import Covariances
from pyriemann.tangentspace import TangentSpace
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

import moabb
from moabb.datasets import BNCI2014_001
from moabb.evaluations import WithinSessionEvaluation
from moabb.paradigms import LeftRightImagery

moabb.set_log_level("info")
mne.set_log_level("CRITICAL")
warnings.filterwarnings("ignore")


###### PIPELINES TO BE COMPARED (do not modify it!) ######
# baseline pipeline to be used to make the comparison, please complete the following line with your framework
pipelines = {}
pipelines["CSP+LDA"] = make_pipeline(CSP(n_components=8), LDA())
pipelines["RG+LR"] = make_pipeline(
    Covariances(), TangentSpace(), LogisticRegression(solver="lbfgs")
)
### Implementation of your framework here: ######
#pipelines["MyPipeline"] =


###### DATASET TO BE USED (do not modify it!) - downloading it the first time can take some time ######
dataset = BNCI2014_001()
subj = [1, 2]
dataset.subject_list = subj


###### DEFINITION OF THE PARADIGM & EVALUATION (do not modify it!) ######
paradigm = LeftRightImagery()
evaluation = WithinSessionEvaluation(
    paradigm=paradigm, datasets=dataset, overwrite=False
)
results = evaluation.process(pipelines)
#print(results.head()) # if you want to look at it...


In [ ]:
###### SCRIPT TO PLOT THE RESULTS - to be updated with your own pipeline #########
# Plot the global distribution of the performance
fig, axes = plt.subplots(1, 2, figsize=[8, 4], sharey=True)

sns.stripplot(
    data=results,
    y="score",
    x="pipeline",
    ax=axes[0],
    jitter=True,
    alpha=0.5,
    zorder=1,
    palette="rocket",
)
sns.pointplot(data=results, y="score", x="pipeline", ax=axes[0], palette="rocket")

axes[0].set_ylabel("ROC AUC")
axes[0].set_ylim(0.5, 1)

paired = results.pivot_table(
    values="score", columns="pipeline", index=["subject", "session"]
)
paired = paired.reset_index()

sns.regplot(data=paired, y="RG+LR", x="CSP+LDA", ax=axes[1], fit_reg=False)
axes[1].plot([0, 1], [0, 1], ls="--", c="k")
axes[1].set_xlim(0.5, 1)

plt.show()

# Plot the individual distribution of the performance
g = sns.catplot(
    kind="bar",
    x="score",
    y="subject",
    hue="pipeline",
    col="dataset",
    height=12,
    aspect=0.5,
    data=results,
    orient="h",
    palette="rocket",
)
plt.show()

## Part 4 - Experimental considerations

## Questions
You plan to launch a new protocol based on EEG acquisitions:
- *What are the two main types of artifacts you may observe? Please indicate one example for each of them.*
- *What are the main steps that compose an EEG processing pipeline?*

You are conducting a Brain-Computer Interface (BCI) experimental protocol consisting of 5 sessions of right-hand motor imagery vs. rest. After the fourth training session, subject Y still shows a global performance of 60%.
Protocol details:

- At each session, the subject was instructed to perform right-hand motor imagery when the visual target was "up" and to remain at rest when the visual target was "down."

- The same features were consistently selected: power spectra in CP3 (10 Hz and 14 Hz) and in C3 (12 Hz and 16 Hz).


*Based on these observations, what suggestions would you propose to help subject Y improve their performance in session 5?*

